In [129]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [130]:
dtypes = pd.Series({'idaviso': np.dtype('uint64'), 'idpostulante': np.dtype('object'),
                   'se_postulo': np.dtype('uint8'), 'sexo_numerico': np.dtype('uint8'),
                   'edad': np.dtype('uint16'), 'estado_code': np.dtype('uint8'),
                   'sexo_code': np.dtype('uint8'), 'nombre_code': np.dtype('uint8'),
                   'nombre_area_code': np.dtype('uint8'),
                    'denominacion_empresa_code': np.dtype('uint16'),
                   'nivel_laboral_code': np.dtype('uint8'),
                   'tipo_de_trabajo_code': np.dtype('uint8'),
                   'nombre_zona_code': np.dtype('uint8')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))


trainingSet = pd.read_csv('Data/trainingSet.csv',dtype=column_types,low_memory=True)
trainingSet.drop(columns=['Unnamed: 0'],inplace=True)

In [131]:
trainingSet.head()

,idaviso,idpostulante,se_postulo,edad,estado_code,sexo_code,nombre_code,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1112248724,NjlD,1,36,2,0,6,120,2468,4,1,7
1,1112286523,ZaO5,1,39,0,0,6,159,952,4,1,7
2,1112272060,ZaO5,1,39,0,0,6,153,36,4,1,7
3,1112288401,ZaO5,1,39,0,0,6,153,3629,4,1,7
4,1112300563,ZaO5,1,39,0,0,6,159,350,4,1,7


In [132]:
trainingSet.info()

#TODO: ENCODEAR TESTING Y TRAINING SETS

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9439964 entries, 0 to 9439963
Data columns (total 12 columns):
idaviso                      uint64
idpostulante                 object
se_postulo                   uint8
edad                         uint16
estado_code                  uint8
sexo_code                    uint8
nombre_code                  uint8
nombre_area_code             uint8
denominacion_empresa_code    uint16
nivel_laboral_code           uint8
tipo_de_trabajo_code         uint8
nombre_zona_code             uint8
dtypes: object(1), uint16(2), uint64(1), uint8(8)
memory usage: 252.1+ MB


# Inicializo el testing set

In [133]:
testingSet = pd.read_csv('TestingSets/testingSet_imp_mean.csv')
testingSet = testingSet.drop(columns='Unnamed: 0')

In [134]:
testingSet.head()

,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code,edad,estado_code,sexo_code,nombre_code,id,idaviso,idpostulante
0,30.0,465.0,1.0,1.0,1.0,41.0,2.0,0.0,0.0,0,739260,6M9ZQR
1,30.0,465.0,1.0,1.0,1.0,30.0,2.0,1.0,5.0,1,739260,6v1xdL
2,30.0,465.0,1.0,1.0,1.0,35.0,2.0,0.0,6.0,2,739260,ezRKm9
3,170.0,3891.0,3.0,1.0,1.0,68.0,2.0,1.0,3.0,3,758580,1Q35ej
4,170.0,3891.0,3.0,1.0,1.0,31.0,1.0,0.0,6.0,4,758580,EAN4J6


TO DO

# Encoding de sets

In [135]:
lb_postulante = LabelEncoder()

In [136]:
a = (testingSet['idpostulante']).astype('str')
ids_postulante = trainingSet['idpostulante'].append(a)

In [137]:
#ids_postulante = trainingSet['idpostulante'].append(str(testingSet['idpostulante']))

In [138]:
lb_postulante.fit(ids_postulante)

LabelEncoder()

In [139]:
trainingSet["idpostulante"] = lb_postulante.transform(trainingSet['idpostulante'])

In [140]:
testingSet["idpostulante"] = lb_postulante.transform(testingSet['idpostulante'])

In [141]:
trainingSet.head()

,idaviso,idpostulante,se_postulo,edad,estado_code,sexo_code,nombre_code,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1112248724,192881,1,36,2,0,6,120,2468,4,1,7
1,1112286523,275438,1,39,0,0,6,159,952,4,1,7
2,1112272060,275438,1,39,0,0,6,153,36,4,1,7
3,1112288401,275438,1,39,0,0,6,153,3629,4,1,7
4,1112300563,275438,1,39,0,0,6,159,350,4,1,7


# Prueba AdaBoost

In [156]:
clf = AdaBoostClassifier(learning_rate=0.1,n_estimators=60,random_state=42)

In [157]:
y_training = trainingSet['se_postulo']

In [158]:
trainingSet2 = trainingSet.drop(columns=['se_postulo']);

In [159]:
trainingSet2;

In [160]:
clf.fit(trainingSet2, y_training)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=60, random_state=42)

In [161]:
clf.score(trainingSet2,y_training)

0.7674009138170442

In [163]:
testingSet2 = testingSet

In [164]:
testingSet2 = testingSet2.drop(columns='idpostulante')

In [165]:
y_testing = rf1.predict(testingSet2)

In [52]:
mse = np.mean(cross_val_score(clf, trainingSet2,y_training,scoring='neg_mean_squared_error', cv=4,  n_jobs=2))

In [53]:
mse

-0.24437379665662057

In [166]:
pred = clf.predict(testingSet2)

# Random Search CV y RandomForests

In [155]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)



{'n_estimators': [5, 15, 26, 36, 47, 57, 68, 78, 89, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [61]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=2, n_jobs = 1)

# Fit the random search model
rf_random.fit(trainingSet2, y_training)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=15.6min
[CV] n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 17.2min remaining:    0.0s


[CV]  n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=16.0min
[CV] n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=16.1min
[CV] n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False, total=22.4min
[CV] n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False, total=22.7min
[CV] n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=35, min_sample

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 166.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=3, n_jobs=1,
          param_distributions={'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=2, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [62]:
rf_random.best_params_

{'n_estimators': 25,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [23]:
mse = np.mean(cross_val_score(rf, trainingSet2,y_training,scoring='neg_mean_squared_error', cv=4,  n_jobs=2))

# Pruebo con los mejores params

In [143]:
rf1 = RandomForestClassifier(n_estimators=5,min_samples_split=10,min_samples_leaf=3,max_features='sqrt',max_depth=4,bootstrap=False)

In [144]:
rf1.fit(trainingSet2,y_training)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=4, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [162]:
#mse = np.mean(cross_val_score(rf1, trainingSet2,y_training,scoring='neg_mean_squared_error', cv=3,  n_jobs=2))

In [ ]:
1

In [145]:
rf1.score(trainingSet2,y_training)

0.7185498800631019

In [60]:
joblib.dump(rf, 'Modelos/random_forest(nest14,minsamp10,minsamleaf4,maxde20).pkl') 

['Modelos/random_forest(nest14,minsamp10,minsamleaf4,maxde20).pkl']

In [146]:
testingSet2 = testingSet

In [149]:
testingSet2 = testingSet2.drop(columns='idpostulante')

In [150]:
y_testing = rf1.predict(testingSet2)

In [151]:
y_testing.sum()

10957

In [152]:
testingSet2.drop(['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code','tipo_de_trabajo_code','nombre_zona_code','edad','estado_code','sexo_code','nombre_code','idaviso',],axis=1,inplace=True)

In [153]:
testingSet2['sepostulo'] = y_testing

In [154]:
testingSet2.to_csv('Submit/predictionADA.csv',index=False)

# KNN

In [61]:
trainingSet.head()

,idaviso,idpostulante,se_postulo,edad,estado_code,sexo_code,nombre_code,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1112248724,192881,1,36,2,0,6,120,2468,4,1,7
1,1112286523,275438,1,39,0,0,6,159,952,4,1,7
2,1112272060,275438,1,39,0,0,6,153,36,4,1,7
3,1112288401,275438,1,39,0,0,6,153,3629,4,1,7
4,1112300563,275438,1,39,0,0,6,159,350,4,1,7


In [77]:
y_training.value_counts()

1    5934281
0    3505683
Name: se_postulo, dtype: int64

In [111]:
KNN = KNeighborsClassifier(n_neighbors=3,weights='distance')

In [112]:
KNN.fit(trainingSet2,y_training)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='distance')

In [113]:
testingSet2 = testingSet

In [114]:
id_testing = testingSet2.id

In [115]:
testingSet2 = testingSet2.drop(columns='id')

In [116]:
KNN.score(trainingSet2,y_training)

1.0

In [84]:
y_testing = KNN.predict(testingSet2)

In [117]:
y_testing.sum()

0

# ExtraTrees

In [89]:
exclf = ExtraTreesClassifier()

In [155]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)



{'n_estimators': [5, 15, 26, 36, 47, 57, 68, 78, 89, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [61]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
exclf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
exclf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=2, n_jobs = 1)

# Fit the random search model
exclf_random.fit(trainingSet2, y_training)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=15.6min
[CV] n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 17.2min remaining:    0.0s


[CV]  n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=16.0min
[CV] n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=30, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=False, total=16.1min
[CV] n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False, total=22.4min
[CV] n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False, total=22.7min
[CV] n_estimators=35, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=35, min_sample

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 166.2min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=3, n_jobs=1,
          param_distributions={'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=2, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [62]:
exclf_random.best_params_

{'n_estimators': 25,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

# Pruebo con los mejores params

In [90]:
testingSet2 = testingSet

In [91]:
id_testing = testingSet2.id

In [103]:
y_training.value_counts()

1    5934281
0    3505683
Name: se_postulo, dtype: int64

In [92]:
testingSet2 = testingSet2.drop(columns='id')

In [98]:
exclf.fit(trainingSet2,y_training)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [99]:
exclf.score(trainingSet2,y_training)

0.999571502603188

In [109]:
y_testing = KNN.predict(testingSet2)

In [110]:
y_testing

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)